In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
tf.__version__

'2.1.0'

### Read the data

First, look in the text:

In [3]:
path_to_file = 'dataset/lovecraft.txt'
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 7831383 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

You, Sir, are the first outsider to be told there is a secret, and split me if I’d have risked tampering that much with—the powers—had ye not been so hot after bygone things.” I shuddered as the man grew colloquial—and with familiar speech of another


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

129 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  "'" :   5,
  '(' :   6,
  ')' :   7,
  ',' :   8,
  '-' :   9,
  '.' :  10,
  '0' :  11,
  '1' :  12,
  '2' :  13,
  '3' :  14,
  '4' :  15,
  '5' :  16,
  '6' :  17,
  '7' :  18,
  '8' :  19,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'pwf  Police g' ---- characters mapped to int ---- > [65 72 55  1  1 39 64 61 58 52 54  1 56]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [9]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

p
w
f
 
 


The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'pwf  Police grilled him mercilessly # while eyes from a hundred worlds looked on  It was a love-trian'
"gle murder that made today's headlines but the answer lay hundreds of thousands of light years away! "
'\none of the basement rooms. \nHe moved slowly and with a kind of painful dignity, as a man moves on hi'
's way to the firing squad. \nA rumpled shock of black hair pointed up the extreme pallor of a gaunt fa'
'ce, empty at the moment of all expression. \nHarsh light from an overhead fixture winked back from tin'


For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [11]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [12]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'pwf  Police grilled him mercilessly # while eyes from a hundred worlds looked on  It was a love-tria'
Target data: 'wf  Police grilled him mercilessly # while eyes from a hundred worlds looked on  It was a love-trian'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step {:4d}".format(i))
  print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 65 ('p')
  expected output: 72 ('w')
Step    1
  input: 72 ('w')
  expected output: 55 ('f')
Step    2
  input: 55 ('f')
  expected output: 1 (' ')
Step    3
  input: 1 (' ')
  expected output: 1 (' ')
Step    4
  input: 1 (' ')
  expected output: 39 ('P')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [14]:
# Batch size
BATCH_SIZE = 512

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((512, 100), (512, 100)), types: (tf.int64, tf.int64)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [15]:
vocab_size = len(vocab) # Length of the vocabulary in chars
embedding_dim = 256 # The embedding dimension
rnn_units = 1024 # Number of RNN units

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, 
                                                           embedding_dim,
                                                           batch_input_shape=[batch_size, None]),
                                 tf.keras.layers.GRU(rnn_units,
                                                     return_sequences=True,
                                                     stateful=True,
                                                     recurrent_initializer='glorot_uniform'),
                                 tf.keras.layers.Dense(vocab_size)])
    return model

In [17]:
chats = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

Please note that we choose to Keras sequential model here since all the layers in the model only have single input and produce single output. In case you want to retrieve and reuse the states from stateful RNN layer, you might want to build your model with Keras functional API or model subclassing. Please check [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse) for more details.

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = chats(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(512, 100, 76) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [19]:
chats.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (512, None, 256)          19456     
_________________________________________________________________
gru (GRU)                    (512, None, 1024)         3938304   
_________________________________________________________________
dense (Dense)                (512, None, 76)           77900     
Total params: 4,035,660
Trainable params: 4,035,660
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [21]:
sampled_indices

array([ 3, 39,  9, 34, 74, 62,  6, 39, 18, 40, 19, 48,  7, 20, 55, 62, 48,
       27, 26, 39, 74, 55, 31, 21,  4, 45, 47, 72, 11, 10, 26, 52, 72,  6,
       35, 60, 53, 38, 28, 73, 19, 30, 11, 17, 52, 29, 57, 60, 44, 57, 69,
       35, 25, 43, 60, 12, 18, 30, 37, 15, 46,  5, 34, 27, 57,  8, 26, 54,
       22, 56,  7, 52, 13, 23, 42, 12, 14, 61, 47,  7, 75, 71, 41, 21,  2,
       37, 47, 15, 52, 73, 39, 23, 66,  5, 63,  8, 75, 41, 38, 13])

Decode these to see the text predicted by this untrained model:

In [22]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '!" \n"Interesting, but quite impossible." \nThe words hardly registered, and Steve babbled on, "We tho'

Next Char Predictions: 
 '"P-Kym(P7Q8Y)9fmYDCPyfH:#VXw0.Ccw(LkdOEx8G06cFhkUhtLBTk17GN4W\'KDh,Ce;g)c2?S13lX)zvR:!NX4cxP?q\'n,zRO2'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [23]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (512, 100, 76)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.3316555


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [24]:
chats.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/home/todd/Documents/ckpts_lovecrafts/training_checkpoints'# Name of the checkpoint files

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                                       save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [26]:
EPOCHS=5 # is 51 the best? loss starts to climb after that

In [27]:
history = chats.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5
2930/2930 [==============================] - 37156s 13s/step - loss: 1.4796
Epoch 2/5
2930/2930 [==============================] - 37158s 13s/step - loss: 1.2477
Epoch 3/5
2930/2930 [==============================] - 37151s 13s/step - loss: 1.2206
Epoch 4/5
2930/2930 [==============================] - 37195s 13s/step - loss: 1.2089
Epoch 5/5
2930/2930 [==============================] - 37121s 13s/step - loss: 1.2027


## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [4]:
#checkpoint_dir = '/home/todd/Documents/ckpts_scifi/training_checkpoints'# Name of the checkpoint files
tf.train.latest_checkpoint(checkpoint_dir)

In [5]:
chats_predict = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

chats_predict.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

chats_predict.build(tf.TensorShape([1, None]))

NameError: name 'build_model' is not defined

In [30]:
chats_predict.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            19456     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 76)             77900     
Total params: 4,035,660
Trainable params: 4,035,660
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.


![To generate text the model's output is fed back to the input](images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [31]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.15

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [32]:
print(generate_text(chats_predict, start_string=u"The desert planet was void of life.\n"))

The desert planet was void of life.
" 
"That was the way the stranger was a member of the computer and the way they were supposed to be any more than a mile away. 
The stars were still staring at the surface of the ship. 
The stars were still staring at the stranger ship and stared at the stranger should have been so close to him. 
He was a little like a construction of the station and the sound of the stars were still strong and so that he was a strange thing to do with the stars. 
The stars were still staring at the side of the stairs. 
The stranger was a man who had been a doctor and a strange thing to do with the stranger's shoulders. 
"I was sure you would have to be a little more than a minute." 
"You mean the computer ship is a complete state of the station of the station, the more interesting that the stranger was a pretty good one. 
The stars were still staring at the stars. 
The stranger was almost in the street and the stars were still staring at the stars. 
The stars were s

The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.

So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point if, for example, to implement _curriculum learning_ to help stabilize the model's open-loop output.

We will use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The procedure works as follows:

* First, reset the RNN state. We do this by calling the `tf.keras.Model.reset_states` method.

* Next, iterate over the dataset (batch by batch) and calculate the *predictions* associated with each.

* Open a `tf.GradientTape`, and calculate the predictions and loss in that context.

* Calculate the gradients of the loss with respect to the model variables using the `tf.GradientTape.grads` method.

* Finally, take a step downwards by using the optimizer's `tf.train.Optimizer.apply_gradients` method.


In [34]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [35]:
optimizer = tf.keras.optimizers.Adam()

In [36]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [37]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # resetting the hidden state at the start of every epoch
  model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.174383640289307
Epoch 1 Batch 100 Loss 2.3455469608306885
Epoch 1 Loss 2.1354
Time taken for 1 epoch 6.386500358581543 sec

Epoch 2 Batch 0 Loss 2.143766403198242
Epoch 2 Batch 100 Loss 1.9540011882781982
Epoch 2 Loss 1.7758
Time taken for 1 epoch 5.211035490036011 sec

Epoch 3 Batch 0 Loss 1.7939282655715942
Epoch 3 Batch 100 Loss 1.697733759880066
Epoch 3 Loss 1.6339
Time taken for 1 epoch 5.233696222305298 sec

Epoch 4 Batch 0 Loss 1.589755654335022
Epoch 4 Batch 100 Loss 1.5560765266418457
Epoch 4 Loss 1.5129
Time taken for 1 epoch 5.385164499282837 sec

Epoch 5 Batch 0 Loss 1.4754937887191772
Epoch 5 Batch 100 Loss 1.4611799716949463
Epoch 5 Loss 1.4449
Time taken for 1 epoch 5.53707480430603 sec

Epoch 6 Batch 0 Loss 1.3802833557128906
Epoch 6 Batch 100 Loss 1.384648084640503
Epoch 6 Loss 1.3626
Time taken for 1 epoch 5.327985763549805 sec

Epoch 7 Batch 0 Loss 1.3318533897399902
Epoch 7 Batch 100 Loss 1.370686650276184
Epoch 7 Loss 1.3535
Time taken for 1 